Installing the google api client for Python

In [ ]:
!pip install google-api-python-client

Importing all the relevant libraries

In [ ]:
from googleapiclient.discovery import build
#add your key to the secrets
from google.colab import userdata
import pandas as pd
import re

Calling the api key added as a variable

In [ ]:
api_key = userdata.get('youtube_api_key')
# API key for YouTube Data API v3
#api_key = "youtube_api_key" #personal api key

# Create a YouTube service object
youtube = build('youtube', 'v3', developerKey=api_key)

The main Function that executes the api call to get the video's and the comments info

In [ ]:
# Function to fetch comments for a YouTube video along with video information
def get_video_comments(video_id, relacion_evento, evento, tipo_evento, condiciones_cuenta): #,max_results = 20000
    # Build a YouTube Data API client
    youtube = build('youtube', 'v3', developerKey=api_key)

    # Fetch video information including duration
    video_response = youtube.videos().list(
        part='snippet,statistics,contentDetails',
        id=video_id
    ).execute()

    # Extract relevant video data
    video_info = video_response['items'][0]['snippet']
    video_stats = video_response['items'][0]['statistics']
    video_content_details = video_response['items'][0]['contentDetails']
    video_data = {
        'video_title': video_info['title'],
        'channel_title': video_info['channelTitle'],
        'video_published_at': video_info['publishedAt'],
        'video_views': video_stats.get('viewCount', 0),
        'video_likes': video_stats.get('likeCount', 0),
        'video_duration': video_content_details['duration'],
        'relacion_evento': relacion_evento,
        'evento': evento,
        'tipo_evento': tipo_evento,
        'condiciones_cuenta': condiciones_cuenta
    }

    # Fetch comments
    comments_data = []
    next_page_token = None
    while True:
        comment_response = youtube.commentThreads().list(
            part='snippet,replies',
            videoId=video_id,
            maxResults=100,  # Maximum results per page (can adjust as needed)
            pageToken=next_page_token
        ).execute()

        # Extract comment data
        for item in comment_response['items']:
            top_comment = item['snippet']['topLevelComment']['snippet']
            comment_data = {
                'comment_id': item['snippet']['topLevelComment']['id'],
                'comment': top_comment['textDisplay'],
                'user_id': top_comment['authorChannelId']['value'],
                'user_name': top_comment['authorDisplayName'],
                'comment_time': top_comment['publishedAt'],
                'comment_likes': top_comment['likeCount'],
                'total_reply_count': item['snippet']['totalReplyCount'],
                'is_top_level_comment': True,
            }
            comment_data.update(video_data)
            comments_data.append(comment_data)

            # Check for replies
            if 'replies' in item:
                for reply in item['replies']['comments']:
                    reply_snippet = reply['snippet']
                    reply_data = {
                        'comment_id': reply['id'],
                        'comment': reply_snippet['textDisplay'],
                        'user_id': reply_snippet['authorChannelId']['value'],
                        'user_name': reply_snippet['authorDisplayName'],
                        'comment_time': reply_snippet['publishedAt'],
                        'comment_likes': reply_snippet['likeCount'],
                        'total_reply_count': 0,  # Replies don't have replies in this structure
                        'is_top_level_comment': False,
                    }
                    reply_data.update(video_data)
                    comments_data.append(reply_data)

        next_page_token = comment_response.get('nextPageToken')
        if not next_page_token:
            break

    return comments_data

List of video information and tags to add to the function iteration

In [ ]:
# List of video IDs and their corresponding tags
videos_info = [
    {"video_id": "eHOu3uRagOc", "relacion_evento": "No", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "awdZ18tP9zw", "relacion_evento": "No", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "qwYz0jO_x24", "relacion_evento": "No", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "M1U2O-uqvDo", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "Pp4BL3zzQTw", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "43Ew_JYSOAg", "relacion_evento": "No", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "3i-dmgxatjc", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "A favor"},
    {"video_id": "82ouEyAkGrI", "relacion_evento": "No", "evento": "24 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "A favor"},
    {"video_id": "6Ol1UIccsU0", "relacion_evento": "No", "evento": "24 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "A favor"},
    {"video_id": "rKIFE9iXvpQ", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "A favor"},
    {"video_id": "nLjtl20j49Q", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "A favor"},
    {"video_id": "lpk5Eko-Kwg", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "A favor"},
    {"video_id": "v4kMHJA0Vls", "relacion_evento": "No", "evento": "Reunion con Elon Musk", "tipo_evento": "Favorable", "condiciones_cuenta": "A favor"},
    {"video_id": "bKgHPHMNa2k", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "En contra"},
    {"video_id": "SQuweBXpvVQ", "relacion_evento": "No", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "En contra"},
    {"video_id": "Ftp-v73Vo5w", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "En contra"},
    {"video_id": "QXoIcF4Dhkc", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "En contra"},
    {"video_id": "czgoifUm0TQ", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "En contra"},
    # Second iteration
    {"video_id": "vHuOaFi3FhU", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "BsyA5POiywg", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "3wf7Xc0G5E8", "relacion_evento": "Si", "evento": "8 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "2XN6Mgpn5KQ", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "gjBlVP5BNzg", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "1FOBFt1DUFU", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "J1g_JWrPnHk", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "FSr0pYqCzQ0", "relacion_evento": "Si", "evento": "Primer paro nacional", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "BP5Pr2CQsdA", "relacion_evento": "Si", "evento": "8 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "KW6lsnTJFnA", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "Eph76rx-GQs", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "fQLCLi6WbKw", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "jEmSacpLtdE", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "lDNkHr9NS7U", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "AM4O1ujZdnI", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "JWBnNEH3NBY", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "iQ3WPb53qp0", "relacion_evento": "Si", "evento": "Marcha federal universitaria", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "N6oDtPCRAj8", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "405cti0GKyo", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "kc7ZFz0teYY", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "qR6GfohNgMA", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "buVTjcub7b0", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "awWE9EO7_uM", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "NG2q3ZH1jcs", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "2wiOYzkcffw", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "8ZeMnr3qzSo", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "kgD9g5fkvEk", "relacion_evento": "Si", "evento": "Crisis diplomatica España", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "NV0DnQHgttg", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "oeI1X6wfY20", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "XWYWG3Eky-M", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "uN16DSUEO9w", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "zWHofogKxjk", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "POf_xCcGu_8", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "ITS2js31zV8", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "HKnIZ8JyLpA", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "04GCLC93DI4", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "5LADv4eBKGs", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "5pnqf4azzdo", "relacion_evento": "Si", "evento": "Crisis alimentos Capital Humano", "tipo_evento": "Desfavorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "mzwW5lZny_k", "relacion_evento": "Si", "evento": "24 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "RnPmihkGPt0", "relacion_evento": "Si", "evento": "24 de marzo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "MB2TkGuqcoI", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "gymK5OKFLcI", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "9S3z-T6rO-A", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "RasymRQzmII", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "XiBPYWPYaTc", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "NRTzEy4lFR8", "relacion_evento": "Si", "evento": "25 de mayo", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "XJCxBJBoe8c", "relacion_evento": "Si", "evento": "20 de junio", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "E0U12cqH0z8", "relacion_evento": "Si", "evento": "20 de junio", "tipo_evento": "Neutral", "condiciones_cuenta": "Neutral"},
    {"video_id": "2xaUOaO9jpo", "relacion_evento": "Si", "evento": "Reunion con Elon Musk", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "FR4oGCOGNy0", "relacion_evento": "Si", "evento": "Reunion con Elon Musk", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "_hAvCeVGnaI", "relacion_evento": "Si", "evento": "Reunion con Elon Musk", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "FvKkVWRWMyo", "relacion_evento": "Si", "evento": "Reunion con Elon Musk", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "yS83u64K3vY", "relacion_evento": "Si", "evento": "Inflación abril", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "njEFWhGEOSg", "relacion_evento": "Si", "evento": "Inflación abril", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "XGmG_TT31XM", "relacion_evento": "Si", "evento": "Inflación abril", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "x19bU9Sdmg4", "relacion_evento": "Si", "evento": "Inflación abril", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "aVRj9ISARRc", "relacion_evento": "Si", "evento": "Inflación abril", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "gcSojKpXKfc", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "aTYWJnkRMtM", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "C_Bi0nBPXWM", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "j_sUR_FlMRM", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "1s2OXJR2pa4", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "rwMFmHSssjY", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "44so2DJWDeY", "relacion_evento": "Si", "evento": "Presentación de libro", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "nAeI5plzCNU", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "qX1uPHpOpT4", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "srPJ8V7wtMo", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "hoRfQL6-Tm8", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "RfGuGKk2vzs", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"},
    {"video_id": "z-TGITUK-u8", "relacion_evento": "Si", "evento": "Aprobación total Ley bases", "tipo_evento": "Favorable", "condiciones_cuenta": "Neutral"}

]

Empty Dataframe creation for the iteration

In [ ]:
# Initialize an empty DataFrame
all_comments_df = pd.DataFrame()

Iteration using the previously defined function and the list of the videos

In [ ]:
for video_info in videos_info:
    video_id = video_info["video_id"]
    relacion_evento = video_info["relacion_evento"]
    evento = video_info["evento"]
    tipo_evento = video_info["tipo_evento"]
    condiciones_cuenta = video_info["condiciones_cuenta"]

    # Fetch comments data for the current video
    comments_data = get_video_comments(video_id, relacion_evento, evento, tipo_evento, condiciones_cuenta)

    # Convert the comments data to a DataFrame
    comments_df = pd.DataFrame(comments_data)

    # Append the current DataFrame to the main DataFrame
    all_comments_df = pd.concat([all_comments_df, comments_df], ignore_index=True)

Saving the dataframe to csv for the further exploration

In [ ]:
# Save the combined DataFrame to a CSV file with UTF-8 encoding
all_comments_df.to_csv('youtube_comments_with_flags_and_replies.csv', index=False, encoding='utf-8')

Traer los comentarios y guardarlos en un csv a partir de una función.